<a href="https://colab.research.google.com/github/JungCesar/bscaithesis/blob/master/bsc_ai_thesis_mfcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -V

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
%%capture
# !pip3 install librosa==0.6.3 numpy soundfile==0.9.0 sklearn PyAudio==0.2.11
!pip3 install librosa numpy soundfile scikit-learn

In [ ]:
# Get access to personal Google Drive account
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# str(path) returns something like: /content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0002.wav
# tqdm is used to create a smart progress bar for the loops, for example it shows loading time

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

from pathlib import Path
from tqdm import tqdm

data = []

for path in tqdm(Path("/content/drive/MyDrive/bsc-ai-thesis/torgo_data").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('/')[-2]
    
    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "filename": name,
            "path": str(path),
            "dysarthria_class": label
        })

    except Exception as e:
        print(str(path), e)
        pass

131it [00:00, 248.59it/s]

/content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0068.wav Failed to open the input "/content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0068.wav" (Invalid data found when processing input).


2000it [00:14, 138.44it/s]


In [ ]:
# Show how the Pandas dataframe looks like currently
import pandas as pd
df = pd.DataFrame(data)
df.head()

,filename,path,dysarthria_class
0,F01_Session1_0006,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
1,F01_Session1_0038,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
2,F01_Session1_0015,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
3,F01_Session1_0024,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
4,F01_Session1_0053,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female


In [ ]:
import os

# Filter broken and non-existed paths
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop(columns='status')

print(f"Step 1: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)

Step 0: 1999
Step 1: 1999


In [ ]:
# Eliminate difference between male and female and print distribbution
df = df.replace({'dysarthria_class' : {'dysarthria_female': 'dysarthria', 'dysarthria_male': 'dysarthria', 'non_dysarthria_female': 'non_dysarthria', 'non_dysarthria_male': 'non_dysarthria'}})
print("Labels: ", df["dysarthria_class"].unique())
print()
df.groupby('dysarthria_class').count()

Labels:  ['non_dysarthria' 'dysarthria']



,filename,path
dysarthria_class,,
dysarthria,999,999
non_dysarthria,1000,1000


In [ ]:
df.head()

,filename,path,dysarthria_class
0,FC02_Session3_0594,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,non_dysarthria
1,FC03_Session2_0137,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,non_dysarthria
2,MC01_Session2_0227,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,non_dysarthria
3,F03_Session2_0035,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria
4,FC02_Session3_0429,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,non_dysarthria


In [ ]:
print(df['path'][0])

/content/drive/MyDrive/bsc-ai-thesis/torgo_data/non_dysarthria_female/FC02_Session3_0594.wav


In [ ]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
import soundfile # to read audio file
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

# everyone takes mean of transpose?
def extract_mfcc(filename):
    y, sr  = librosa.load(filename)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=30).T, axis=0)
    return mfcc

# def load_data(test_size=0.2):
#     X, y = [], []
#     df = data.reset_index()  # make sure indexes pair with number of rows
#     for _, row in df.iterrows():
#         # print(row['path'], row['dysarthria_class'])
#         # get the base name of the audio file
#         # basename = os.path.basename(file)
#         # extract speech features
#         features = extract_mfcc(row['path'])
#         # get the emotion label
#         label = row['dysarthria_class']
#         # add to data
#         X.append(features)
#         y.append(label)
#     # split the data to training and testing and return it
#     return train_test_split(np.array(X), y, test_size=test_size)

In [ ]:
# data, sr = librosa.load('/content/drive/MyDrive/bsc-ai-thesis/torgo_data/non_dysarthria_male/MC04_Session1_0444.wav')
# print(data.shape)

In [ ]:
test_size=0.2
X, y = [], []
df = df.reset_index()  # make sure indexes pair with number of rows
for _, row in df.iterrows():
    # print(row['path'], row['dysarthria_class'])
    # get the base name of the audio file
    # basename = os.path.basename(file)
    # extract speech features
    features = extract_mfcc(row['path'])
    # get the emotion label
    label = row['dysarthria_class']
    # add to data
    X.append(features)
    y.append(label)

tts = train_test_split(np.array(X), y, test_size=test_size)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=946
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=185
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=581
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=209
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=214
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=583
  warnings.warn(


In [ ]:
np.array(X).shape

(1999, 30)

In [ ]:
print(type(X))

<class 'list'>


In [ ]:
# load RAVDESS dataset, 75% training 25% testing
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.2)

In [ ]:
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using extract_features() function
print("[+] Number of features:", X_train.shape[1])

[+] Number of training samples: 1599
[+] Number of testing samples: 400
[+] Number of features: 30


In [ ]:
# best model, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}

In [ ]:
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model

# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
# Do this in Keras or PyTorch
model = MLPClassifier(**model_params)

In [ ]:
# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

[*] Training the model...


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
from sklearn.metrics import accuracy_score # to measure how good we are

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 96.50%
